In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime

#load files
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
macro = pd.read_csv('../input/macro.csv')
id_test = test.id

#multiplier = 0.969
# truncate the extreme values in price_doc #
Q1 = np.percentile(np.log1p(train.price_doc.values), 25)
Q2 = np.percentile(np.log1p(train.price_doc.values), 50)
Q3 = np.percentile(np.log1p(train.price_doc.values), 75)
IQR=Q3-Q1
infbdd=Q1-1.5 * IQR 
supbdd=Q3+1.5 * IQR 
train['price_doc'].ix[train['price_doc']>int(2.5*np.exp(supbdd))] = int(2.5*np.exp(supbdd))
train['price_doc'].ix[train['price_doc']<int(np.exp(13.5))] = int(np.exp(13.5))


#clean data


#equal_index = [601,1896,2791]
#test.ix[equal_index, "life_sq"] = test.ix[equal_index, "full_sq"]

#bad_index = test[test.life_sq > test.full_sq].index
#test.ix[bad_index, "life_sq"] = np.NaN

bad_index = train[train.life_sq < 2].index
train.ix[bad_index, "life_sq"] = np.NaN

#bad_index = test[test.life_sq < 5].index
#test.ix[bad_index, "life_sq"] = np.NaN

bad_index = train[train.full_sq < 2].index
train.ix[bad_index, "full_sq"] = np.NaN
###

#bad_index = test[test.full_sq < 5].index
#test.ix[bad_index, "full_sq"] = np.NaN

#kitch_is_build_year = [13117]
#train.ix[kitch_is_build_year, "build_year"] = train.ix[kitch_is_build_year, "kitch_sq"]
#bad_index = train[train.kitch_sq >= train.life_sq].index
#train.ix[bad_index, "kitch_sq"] = np.NaN

#bad_index = test[test.kitch_sq >= test.life_sq].index
#test.ix[bad_index, "kitch_sq"] = np.NaN

#bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
#train.ix[bad_index, "kitch_sq"] = np.NaN

#bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
#test.ix[bad_index, "kitch_sq"] = np.NaN




#bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
#test.ix[bad_index, "full_sq"] = np.NaN


#bad_index = test[test.life_sq > 200].index
#test.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN

In [ ]:
expensive = np.nanpercentile((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values)), 99.9)
cheap = np.nanpercentile((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values)), 0.1)
print(expensive)
print(cheap)

In [ ]:
ulimit = np.nanpercentile((train.life_sq.values)/(train.full_sq.values), 99)
llimit = np.nanpercentile((train.life_sq.values)/(train.full_sq.values), 1)
median = np.nanpercentile((train.life_sq.values)/(train.full_sq.values), 50)
mean=np.nanmean((train.life_sq.values)/(train.full_sq.values))
print(ulimit)
print(llimit)
print(mean)

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=(train.full_sq.values), y=(train.life_sq.values), size=10)
plt.ylabel('life_sq', fontsize=12)
plt.xlabel('full_sq', fontsize=12)
plt.show()

In [ ]:

life_bad_index=train[(train.life_sq > train.full_sq) & ((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))>3)&((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))<5.25)].index
train.ix[life_bad_index, "life_sq"] = train.ix[life_bad_index,"full_sq"]*0.85
full_bad_index=train[(train.life_sq > train.full_sq)&((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values))>3)&((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values))<5.25)].index
train.ix[full_bad_index, "full_sq"] = train.ix[full_bad_index,"life_sq"]/0.85
true_bad_index = train[train.life_sq > train.full_sq]
true_bad_index

In [ ]:
full_bad_index = train[(train.life_sq / train.full_sq < 0.39)&((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values))<5.25)&((np.log1p(train.price_doc.values)/np.log1p(train.life_sq.values))>3)].index
train.ix[full_bad_index, "full_sq"] = train.ix[full_bad_index,"life_sq"]/0.85

life_bad_index=train[(train.life_sq / train.full_sq < 0.39)&((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))<5.25)&((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))>3)].index
train.ix[life_bad_index, "life_sq"] = train.ix[life_bad_index,"full_sq"]*0.85

true_bad_index=train[(train.life_sq / train.full_sq < 0.39)]
true_bad_index

In [ ]:
bad_index = train[((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))<2.7)&(train.full_sq>200)].index
train.ix[bad_index, "full_sq"] = np.NaN
train.ix[bad_index, "life_sq"] = np.NaN

In [ ]:
bad_index = train[((np.log1p(train.price_doc.values*10)/np.log1p(train.full_sq.values))<3)]
bad_index

In [ ]:
bad_index = train[(train.life_sq==14)&(train.price_doc==7000000)]
bad_index

In [ ]:
expensive = np.nanpercentile((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values)), 99.9)
cheap = np.nanpercentile((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values)), 0.1)
print(expensive)
print(cheap)

In [ ]:
bad_index = train[((np.log1p(train.price_doc.values)/np.log1p(train.full_sq.values))>5.3)]
bad_index
#train.ix[bad_index, "full_sq"] = np.NaN

In [ ]:
np.log1p(10)

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=np.log1p(train.full_sq.values), y=np.log1p(train.price_doc.values), size=10)
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Log of Total area in square metre', fontsize=12)
plt.show()

In [ ]:
plt.figure(figsize=(12,12))
sns.jointplot(x=np.log1p(train.life_sq.values), y=np.log1p(train.price_doc.values), 
              kind='kde', size=10)
plt.ylabel('Log of Price', fontsize=12)
plt.xlabel('Log of living area in square metre', fontsize=12)
plt.show()